In [1]:
import sys
sys.path.append("..")

In [2]:
from constants import *
from supporting_functions import *
from des_functions import *
from cryptanalysis import *
from collections import Counter

In [3]:
key = format(random.randint(0,2**56-1), '056b')
key = add_parity_key(key)
keys = compute_keys(key)
key

'1000100_0011010_1101110_0101000_1000101_0101101_1111111_0100011_'

In [4]:
# CHARACTERISTIC + 1 Round S-Boxes who hav 0 as input
exp = expansion(THREE_ROUND_CHAR1_OP_XOR[32:])
zero_ix = []
s_box_op = ""
for i in range(8):
    if exp[6*i:6*(i+1)]=='0'*6:
        zero_ix.append(i)
        s_box_op+='0'*4
    else:
        s_box_op+='1'*4
l_prev_mask = permutation(s_box_op)
l_prev_mask = compute_xor(l_prev_mask, '1' * len(l_prev_mask))

In [5]:
# PLAIN TEXT
plain_texts = []
plain_pairs = []
for i in range(int(1e4)):
    pair = get_pair(THREE_ROUND_CHAR1_IP_XOR)
    input_pair = [bits_to_str(inv_initial_permutation(str_to_bits(pair[0]))),
                    bits_to_str(inv_initial_permutation(str_to_bits(pair[1])))]
    plain_pairs.append(input_pair)
    plain_texts.append(input_pair[0])
    plain_texts.append(input_pair[1])
    
# ENCODE
cipher_texts = des_encode_multiple(plain_texts)

# PAIRS
cipher_pairs = []
for i, c in enumerate(cipher_texts):
    c_old = initial_permutation(str_to_bits(c))
    c_new = c_old[32:] + c_old[:32]
    if i%2==0:
        cipher_pairs.append([c_new])
    else:
        cipher_pairs[-1].append(c_new)
        
output_xors = []
for pair in cipher_pairs:
    ans = compute_xor(pair[0], pair[1])
    output_xors.append(ans)

# S-BOX OUTPUT_XOR
s_box_exp_op_xors = []
s_box_ip_pairs = []
for i,op_xor in enumerate(output_xors):
    l_op_xor = op_xor[:32]
    r_op_xor = op_xor[32:]
    r_op_xor = inv_permutation(r_op_xor)
    s_box_xor = r_op_xor
    s_box_exp_op_xors.append(s_box_xor)
    
    r_prev_pair = cipher_pairs[i]
    r_prev_pair = r_prev_pair[0][:32], r_prev_pair[1][:32]
    r_prev_pair = expansion(r_prev_pair[0]), expansion(r_prev_pair[1])
    s_box_ip_pairs.append(r_prev_pair)
    
   
# TESTING FOR EACH BYTE AND EACH KEY
counter = np.zeros([8,64])
for j in range(64):
    candidate_key = format(j, '06b') * 8
    for k in range(len(s_box_exp_op_xors)):
        s_box_op_xor = s_box_exp_op_xors[k]
        s_box_ip_pair = s_box_ip_pairs[k]
        s_box_ip_pair = compute_xor(s_box_ip_pair[0], candidate_key), compute_xor(s_box_ip_pair[1], candidate_key)
        s_box_ip_xor = compute_xor(s_box_ip_pair[0], s_box_ip_pair[1])
        s_box_op_pair = s_boxes_encode(s_box_ip_pair[0]), s_boxes_encode(s_box_ip_pair[1])
        
        masked_exp_op_xor = compute_and(l_prev_mask, s_box_op_xor)
        masked_true_op_xor = compute_and(l_prev_mask, compute_xor(s_box_op_pair[0], s_box_op_pair[1]))
        
        # Rejecting those for which output xor is not possible
        for i in range(8):
            ip_xor = s_box_ip_xor[6*i:6*(i+1)]
            op_xor = s_box_op_xor[4*i:4*(i+1)]
            if op_xor not in ip_xor_to_op_xor[i][ip_xor]:
                continue
            
            if masked_exp_op_xor[4*i:4*(i+1)]==masked_true_op_xor[4*i:4*(i+1)]:
                counter[i][j]+=1
    
for i in range(8):
    predicted_ix = np.argmax(counter[i])
    counter[i].sort()
    print(counter[i])
    print(format(predicted_ix, '06b'))

[1061. 1077. 1077. 1077. 1089. 1092. 1093. 1098. 1099. 1101. 1101. 1101.
 1104. 1105. 1106. 1106. 1110. 1114. 1114. 1117. 1118. 1120. 1121. 1121.
 1123. 1124. 1125. 1126. 1126. 1129. 1131. 1131. 1131. 1133. 1134. 1134.
 1135. 1136. 1139. 1139. 1140. 1143. 1143. 1145. 1147. 1148. 1150. 1152.
 1154. 1155. 1155. 1156. 1156. 1158. 1160. 1161. 1162. 1162. 1165. 1168.
 1176. 1178. 1180. 1604.]
101101
[1165. 1166. 1187. 1189. 1190. 1192. 1192. 1199. 1206. 1213. 1213. 1214.
 1214. 1215. 1216. 1216. 1221. 1223. 1230. 1231. 1236. 1239. 1242. 1244.
 1245. 1245. 1249. 1249. 1253. 1254. 1266. 1269. 1270. 1276. 1282. 1283.
 1284. 1285. 1288. 1289. 1291. 1297. 1298. 1300. 1301. 1315. 1319. 1319.
 1320. 1328. 1343. 1346. 1356. 1357. 1363. 1363. 1388. 1399. 1402. 1405.
 1414. 1438. 1440. 3464.]
111011
[1929. 1933. 1944. 1972. 1982. 1988. 1989. 1993. 1996. 1997. 2000. 2000.
 2002. 2007. 2012. 2013. 2014. 2016. 2017. 2018. 2024. 2026. 2027. 2027.
 2032. 2034. 2036. 2040. 2045. 2046. 2049. 2052. 2053. 205